In [26]:
import pandas as pd
import os

from glob import glob
from commons import load_paths_images_sorted, plot, check_colision_border

from skimage.io import imread_collection, imread, imsave
from skimage.measure import label as region_label, regionprops_table, regionprops

from skimage import (
    img_as_ubyte,
)

In [27]:
def extrat_area(clusters):
    
    return max(map(lambda item: item.area, regionprops(clusters)))

def select_only_valid_masks(image_paths:list):
    images = []
    paths = []
    
    for (img, path) in zip(imread_collection(image_paths), image_paths):
        if not check_colision_border(img):
            images.append(img)
            paths.append(path)
        
    return {
        'images': images,
        'paths': paths
    }

In [31]:
_input = 'sobel_canny'

In [32]:
for index_image in glob(f'{_input}/*'):
    
    image_paths = load_paths_images_sorted(index_image)
    images_paths = select_only_valid_masks(image_paths)
    images_area_pd = pd.DataFrame({
        'area': map(extrat_area, images_paths['images']),
        'path': images_paths['paths']
    })

    areas = images_area_pd.area.to_list()
    median = areas[len(areas) // 2] #median = images_area_pd.area.median()
    query = images_area_pd.area == median
    best_mask, *_ = images_area_pd[query].path.to_list()
    
    image_name = index_image.split("/")[1]
    imsave(f'auto_selected_masks/{image_name}.tif', img_as_ubyte(imread(best_mask)))

100%|█████████████████████████████████████████████████████████████| 72/72 [00:00<00:00, 669600.64it/s]
